# Feature Extraction using FAST

In [2]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
import glob

from sklearn.model_selection import train_test_split

In [3]:
input0 = './train/Preprocessed/'
temp = ['Bowlegs', 'Knock Knees', 'Normal Knee']

In [13]:
count = 0
for i in temp:
    for filename in os.listdir(input0 + i):
        img = cv2.resize(cv2.imread(input0 + i + '/' + filename), (512, 512))#resize image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #initialise sift descriptor
        fast = cv2.xfeatures2d.StarDetector_create()
        keypoints = fast.detect(gray, None)
        brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
        descriptors = brief.compute(gray, None)
        
        #convert the descriptor array into a dataframe format
        out=pd.DataFrame(descriptors)
        
        #append to the csv file
        csv_data=out.to_csv('./FAST/FAST_' + i + '.csv', mode='a', index=False)
        count += 1
    print(i + ": " + str(count))

Bowlegs: 111
Knock Knees: 360
Normal Knee: 609


# K-means

In [11]:
data1 = pd.read_csv('./FAST/FAST_Bowlegs.csv', dtype='uint8')
data2 = pd.read_csv('./FAST/FAST_Knock Knees.csv', dtype='uint8')
data3 = pd.read_csv('./FAST/FAST_Normal Knee.csv', dtype='uint8')

data1 = data1.astype('uint8')
data2 = data2.astype('uint8')
data3 = data3.astype('uint8')

ParserError: Error tokenizing data. C error: Expected 58 fields in line 4, saw 79


In [8]:
#performing kmeans on each class
#Bowlegs
kmeans1 = KMeans(n_clusters=5)
kmeans1.fit(data1)

#Knock Knees
kmeans2 = KMeans(n_clusters=5)
kmeans2.fit(data2)

#Normal
kmeans3 = KMeans(n_clusters=5)
kmeans3.fit(data3)

KMeans(n_clusters=5)

In [9]:
hist1=np.histogram(kmeans1.labels_,bins=[0,1,2,3,4,5])
hist2=np.histogram(kmeans2.labels_,bins=[0,1,2,3,4,5])
hist3=np.histogram(kmeans3.labels_,bins=[0,1,2,3,4,5])

print('histogram of bowed')
print(hist1,"\n")

print('histogram of knocked')
print(hist2,"\n")

print('histogram of normal')
print(hist3,"\n")

histogram of bowed
(array([3714, 2468, 3072, 4192, 4394], dtype=int64), array([0, 1, 2, 3, 4, 5])) 

histogram of knocked
(array([17271, 14975,  7585,  5175, 24516], dtype=int64), array([0, 1, 2, 3, 4, 5])) 

histogram of normal
(array([14837, 11392, 19722,  7531, 11596], dtype=int64), array([0, 1, 2, 3, 4, 5])) 



In [ ]:
#performing kmeans prediction of the entire apple dataset with the pretrained kmeans model
#initialising i=0; as its the first class
i=0
for j in temp:
    data=[]
    for filename in os.listdir(input0 + j):
        path = input0 + j + '/' + filename
        a=cv2.imread(path)
        resize=(512,512)
        img=cv2.resize(a,resize)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #initialise sift descriptor
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        
        #convert the descriptor array into a dataframe format
        out=pd.DataFrame(descriptors)
        
        array_double = np.array(out, dtype=np.double)

        a=kmeans1.predict(array_double)
        hist=np.histogram(a,bins=[0,1,2,3,4,5])
        #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
        data.append(hist[0])
    
    #convert Array to Dataframe and append to the list
    Output = pd.DataFrame(data)
    #add row class 
    Output["Class"] = i 
    csv_data=Output.to_csv('./SIFT/SIFT_'+ j + 'Final.csv', mode='a', index=False)
    i += 1

In [14]:
final = pd.read_csv('./SIFT/SIFT_BowlegsFinal.csv')
temp1 = final.append(pd.read_csv('./SIFT/SIFT_Knock KneesFinal.csv'))
tc = pd.read_csv('./SIFT/SIFT_Normal KneeFinal.csv')
tcc = temp1.append(tc)
csv_data = tcc.to_csv('./SIFT/SIFT_FeatureFinal.csv', mode='a', index=False)
print(tcc)

      0   1   2   3   4  Class
0     4   3   2  24   5      0
1     5   8   1  18   3      0
2     6   8  34  28  15      0
3    14  38  52  60  39      0
4     8  41  39  69  33      0
..   ..  ..  ..  ..  ..    ...
244  37  28   5  30  11      2
245   7   4  16  16   1      2
246   9   4  18  18   2      2
247  12   5  23  15   1      2
248  16   8  20  21   6      2

[609 rows x 6 columns]


C:\Users\sumit\AppData\Local\Temp\ipykernel_7652\555719870.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp1 = final.append(pd.read_csv('./SIFT/SIFT_Knock KneesFinal.csv'))
C:\Users\sumit\AppData\Local\Temp\ipykernel_7652\555719870.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tcc = temp1.append(tc)


# Data spliting

In [15]:
df = pd.read_csv('./SIFT/SIFT_FeatureFinal.csv')
X_train = df.iloc[:, 0:5]
Y_train = df.iloc[:, 5:6]
train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train, 
                                                      test_size=0.5, 
                                                      stratify=Y_train, 
                                                      random_state=8)

# KNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

knn = KNeighborsClassifier(n_neighbors=7)
  
knn.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=7)

In [17]:
print('\nResults obtained for KNN')

# accuracy on training data
y_pred_t = knn.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = knn.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))



Results obtained for KNN

Results obtained on Training Data
Accuracy on Train data :  0.7171052631578947
Precision:  0.689380972757096
Recall:  0.6774193548387096
F-score:  0.6820155747044955

Results obtained on Testing Data
Accuracy on Test data :  0.5672131147540984
Precision:  0.5270189986118522
Recall:  0.5258181818181819
F-score:  0.5253049361539928


# XgBoost

In [18]:
import xgboost as xgb

classifier = xgb.XGBClassifier(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

classifier.fit(train_x, train_y)

[23:26:45] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\sumit\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
c:\Users\sumit\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sumit\a

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [19]:
print('\nResults obtained for XgBoost')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for XgBoost

Results obtained on Training Data
Accuracy on Train data :  1.0
Precision:  1.0
Recall:  1.0
F-score:  1.0

Results obtained on Testing Data
Accuracy on Test data :  0.6426229508196721
Precision:  0.6093477373129895
Recall:  0.5837575757575758
F-score:  0.5891450433469241


# SVM

In [20]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

subclassifier = SVC(kernel='rbf')
classifier = OneVsOneClassifier(estimator=subclassifier)
classifier.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


OneVsOneClassifier(estimator=SVC())

In [21]:
print('\nResults obtained for SVM')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for SVM

Results obtained on Training Data
Accuracy on Train data :  0.5789473684210527
Precision:  0.3856209150326797
Recall:  0.4731182795698925
F-score:  0.423744292237443

Results obtained on Testing Data
Accuracy on Test data :  0.5377049180327869
Precision:  0.3600370762711864
Recall:  0.4373333333333333
F-score:  0.3923862872898289


c:\Users\sumit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes

In [22]:
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB()
classifier.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BernoulliNB()

In [23]:
print('\nResults obtained for Naive Bayes')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for Naive Bayes

Results obtained on Training Data
Accuracy on Train data :  0.4144736842105263
Precision:  0.4701986754966887
Recall:  0.3387096774193548
F-score:  0.2046352187197258

Results obtained on Testing Data
Accuracy on Test data :  0.4163934426229508
Precision:  0.3873200442967885
Recall:  0.33866666666666667
F-score:  0.2095570841067074


c:\Users\sumit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
